In [1]:
import sys
sys.path.append("../")
import pandas as pd
import plotly.graph_objects as go
import datetime as dt
from plotting import CandlePlot

In [2]:
from infrastructure.instrument_collection import instrumentCollection as ic

In [3]:
pair = "GBP_JPY"
granularity = "H4"
df = pd.read_pickle(f"../data/{pair}_{granularity}.pkl")
MA_LIST = [10, 20, 50, 100, 200]

In [4]:
df_ma = df[['time','mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [5]:
df_ma.head(10)

,time,mid_o,mid_h,mid_l,mid_c
0,2022-06-09 05:00:00+00:00,168.110,168.126,166.692,167.401
1,2022-06-09 09:00:00+00:00,167.402,168.098,166.955,167.660
2,2022-06-09 13:00:00+00:00,167.659,168.617,167.581,167.900
3,2022-06-09 17:00:00+00:00,167.899,168.080,167.640,167.913
4,2022-06-09 21:00:00+00:00,167.859,168.044,167.360,167.506
5,2022-06-10 01:00:00+00:00,167.507,167.716,167.340,167.711
6,2022-06-10 05:00:00+00:00,167.714,167.722,166.552,167.091
7,2022-06-10 09:00:00+00:00,167.091,167.156,165.786,165.888
8,2022-06-10 13:00:00+00:00,165.890,166.294,165.174,165.360
9,2022-06-10 17:00:00+00:00,165.360,165.579,165.224,165.534


In [6]:
for ma in MA_LIST:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(inplace=True, drop=True)

In [7]:
df_plot = df_ma.iloc[:500]

In [8]:
cp = CandlePlot(df_plot)

In [9]:
traces = [ f"MA_{x}" for x in MA_LIST ]

In [10]:
cp.show_plot(line_traces=traces)

In [11]:
MA_S = "MA_10"
MA_L = "MA_20"
BUY = 1
SELL = -1
NONE = 0

In [12]:
df_an = df_ma[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', MA_S, MA_L]].copy()

In [13]:
df_an.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_10,MA_20
0,2022-07-26 09:00:00+00:00,164.322,164.340,163.476,163.856,164.2242,164.40130
1,2022-07-26 13:00:00+00:00,163.855,164.636,163.582,164.293,164.3197,164.33840
2,2022-07-26 17:00:00+00:00,164.292,164.795,164.254,164.754,164.4727,164.28270
3,2022-07-26 21:00:00+00:00,164.658,165.174,164.654,165.098,164.5757,164.29430
4,2022-07-27 01:00:00+00:00,165.103,165.226,164.892,165.225,164.6158,164.31335


In [14]:
df_an['DELTA'] = df_an.MA_10 - df_an.MA_20
df_an['DELTA_PREV'] = df_an.DELTA.shift(1)

In [15]:
def is_trade(row):
    if row.DELTA >= 0 and row.DELTA_PREV < 0:
        return BUY
    elif row.DELTA <= 0 and row.DELTA_PREV > 0:
        return SELL
    return NONE

In [16]:
df_an['TRADE'] = df_an.apply(is_trade, axis=1)

In [17]:
df_trades = df_an[df_an.TRADE != NONE].copy()

In [18]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_10,MA_20,DELTA,DELTA_PREV,TRADE
2,2022-07-26 17:00:00+00:00,164.292,164.795,164.254,164.754,164.4727,164.28270,0.19000,-0.01870,1
15,2022-07-28 21:00:00+00:00,163.504,163.907,163.454,163.748,164.5350,164.58770,-0.05270,0.02150,-1
38,2022-08-03 17:00:00+00:00,162.862,163.014,162.594,162.606,161.7332,161.62905,0.10415,-0.13545,1
50,2022-08-05 17:00:00+00:00,163.214,163.336,162.808,162.967,162.1507,162.22445,-0.07375,0.04070,-1
53,2022-08-08 05:00:00+00:00,163.344,163.850,163.185,163.309,162.4570,162.41630,0.04070,-0.08600,1


In [19]:
cp = CandlePlot(df_an.iloc[10: 500])
cp.show_plot(line_traces=[MA_S, MA_L])

In [20]:
ic.LoadInstruments("../data")

In [21]:
ic.instruments_dict[pair]

{'name': 'GBP_JPY', 'ins_type': 'CURRENCY', 'displayName': 'GBP/JPY', 'pipLocation': 0.01, 'tradeUnitsPrecision': 0, 'marginRate': 0.0333}

In [22]:
ins_data = ic.instruments_dict[pair]

In [23]:
df_trades.head(20)

,time,mid_o,mid_h,mid_l,mid_c,MA_10,MA_20,DELTA,DELTA_PREV,TRADE
2,2022-07-26 17:00:00+00:00,164.292,164.795,164.254,164.754,164.4727,164.28270,0.19000,-0.01870,1
15,2022-07-28 21:00:00+00:00,163.504,163.907,163.454,163.748,164.5350,164.58770,-0.05270,0.02150,-1
38,2022-08-03 17:00:00+00:00,162.862,163.014,162.594,162.606,161.7332,161.62905,0.10415,-0.13545,1
50,2022-08-05 17:00:00+00:00,163.214,163.336,162.808,162.967,162.1507,162.22445,-0.07375,0.04070,-1
53,2022-08-08 05:00:00+00:00,163.344,163.850,163.185,163.309,162.4570,162.41630,0.04070,-0.08600,1
68,2022-08-10 17:00:00+00:00,162.350,162.768,162.076,162.346,163.0161,163.06560,-0.04950,0.03740,-1
95,2022-08-17 05:00:00+00:00,162.583,163.392,162.482,163.174,161.8321,161.71250,0.11960,-0.06960,1
108,2022-08-19 09:00:00+00:00,162.349,162.378,161.862,162.108,162.3534,162.43640,-0.08300,0.04540,-1
136,2022-08-26 01:00:00+00:00,161.478,161.720,161.448,161.578,161.6044,161.57875,0.02565,-0.03095,1
156,2022-08-31 09:00:00+00:00,161.328,161.534,160.899,161.292,161.8226,161.85945,-0.03685,0.05555,-1


In [24]:
df_trades['DIFF'] = df_trades.mid_c.diff().shift(-1)
df_trades.fillna(0, inplace=True)

In [25]:
df_trades['GAIN'] = df_trades['DIFF'] / ins_data.pipLocation
df_trades['GAIN'] = df_trades['GAIN'] * df_trades['TRADE']

In [26]:
df_trades.GAIN.sum(numeric_only=True)

np.float64(138.6000000000056)

In [27]:
df_trades['GAIN_C'] = df_trades['GAIN'].cumsum()

In [28]:
cp = CandlePlot(df_trades, candles=False)
cp.show_plot(line_traces=['GAIN_C'])